<a href="https://colab.research.google.com/github/sugarforever/LangChain-Advanced/blob/main/Retrievers/01_MultiQuery_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.8/437.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml)

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "your valid openai api key"

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load blog post
loader = WebBaseLoader("https://blog.langchain.dev/langchain-prompt-hub/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "What can we do with LangChain hub?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(), llm=llm)

In [ ]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can the LangChain hub be utilized?', '2. What are the possible applications of the LangChain hub?', '3. In what ways can the LangChain hub be used effectively?']


[Document(page_content="Today, we're excited to launch LangChain Hub‚Äìa home for uploading, browsing, pulling, and managing your prompts. (Soon, we'll be adding other artifacts like chains and agents).\uf8ffüí°Explore the Hub hereLangChain Hub is built into LangSmith (more on that below) so there are 2 ways to start exploring LangChain Hub.With LangSmith access: Full read and write permissions. You can explore all existing prompts and upload your own by logging in and navigate to the Hub from your admin panel.Without", metadata={'language': 'en', 'source': 'https://blog.langchain.dev/langchain-prompt-hub/', 'title': 'Announcing LangChain Hub'}),
 Document(page_content="way to facilitate this kind of collaboration.We're aiming to make LangChain Hub the best place for teams to write and manage prompts, together. The product isn't quite there today‚Äìthis first iteration only supports personal accounts‚Äìbut we're actively looking for organizations that are excited to explore an Alpha wi

In [ ]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.retrievers.multi_query import LineListOutputParser


# Output parser will split the LLM result into a list of queries
'''
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        print(f"Input text: {text}")
        lines = text.strip().split("\n")
        return LineList(lines=lines)
'''

output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

In [ ]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

unique_docs = retriever.get_relevant_documents(query=question)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can the LangChain hub be utilized?', '2. What are the possible applications of the LangChain hub?', '3. In what ways can the LangChain hub be used?', '4. What functionalities does the LangChain hub offer?', '5. What are the potential uses of the LangChain hub?']


[Document(page_content="Today, we're excited to launch LangChain Hub‚Äìa home for uploading, browsing, pulling, and managing your prompts. (Soon, we'll be adding other artifacts like chains and agents).\uf8ffüí°Explore the Hub hereLangChain Hub is built into LangSmith (more on that below) so there are 2 ways to start exploring LangChain Hub.With LangSmith access: Full read and write permissions. You can explore all existing prompts and upload your own by logging in and navigate to the Hub from your admin panel.Without", metadata={'language': 'en', 'source': 'https://blog.langchain.dev/langchain-prompt-hub/', 'title': 'Announcing LangChain Hub'}),
 Document(page_content="way to facilitate this kind of collaboration.We're aiming to make LangChain Hub the best place for teams to write and manage prompts, together. The product isn't quite there today‚Äìthis first iteration only supports personal accounts‚Äìbut we're actively looking for organizations that are excited to explore an Alpha wi